In [17]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas.api.types import is_string_dtype, is_numeric_dtype
from scipy import stats
from itertools import permutations
from mlxtend.preprocessing import TransactionEncoder

# Data sets

In [18]:
# Read datasets
trans = pd.read_csv('/Users/hafanhh/Desktop/Market Basket Analysis in Python/bookstore_transactions.csv')
movies = pd.read_csv('https://assets.datacamp.com/production/repositories/5654/datasets/292498b8454c35e12e9be7ef18bc53c89b3bd073/movielens_movies.csv')
retail = pd.read_csv('https://assets.datacamp.com/production/repositories/5654/datasets/5a3bc2ebccb77684a6d8a9f3fbec23fe04d4e3aa/online_retail.csv')

print(trans.head())
print(movies.head())
print(retail.head())

          Transaction
0    History,Bookmark
1    History,Bookmark
2    Fiction,Bookmark
3  Biography,Bookmark
4    History,Bookmark
   movieId                               title  \
0        1                    Toy Story (1995)   
1        2                      Jumanji (1995)   
2        3             Grumpier Old Men (1995)   
3        4            Waiting to Exhale (1995)   
4        5  Father of the Bride Part II (1995)   

                                        genres  
0  Adventure|Animation|Children|Comedy|Fantasy  
1                   Adventure|Children|Fantasy  
2                               Comedy|Romance  
3                         Comedy|Drama|Romance  
4                                       Comedy  
  InvoiceNo StockCode                          Description
0    562583    35637A      IVORY STRING CURTAIN WITH POLE 
1    562583    35638A        PINK AND BLACK STRING CURTAIN
2    562583    84927F                PSYCHEDELIC TILE HOOK
3    562583     22425                

# 1. Book Transaction datatset

### 1.1. Cross-selling products

The small grocery store has decided to cross-sell chewing gum with either coffee, cereal, or bread. To determine which of the three items is best to use, the store owner has performed an experiment. For one week, she sold chewing gum next to the register and recorded all transactions where it was purchased with either coffee, cereal, or bread. 

In [19]:
trans.value_counts()

Transaction       
Biography,Bookmark    40
Fiction,Bookmark      25
History,Bookmark      25
Poetry,Bookmark        9
Name: count, dtype: int64

* Identifying association rules

In [20]:
# Split transaction strings into lists
transactions = trans['Transaction'].apply(lambda t: t.split(','))

# Convert Df into list of strings
transactions = list(transactions)
print(transactions[:5])

[['History', 'Bookmark'], ['History', 'Bookmark'], ['Fiction', 'Bookmark'], ['Biography', 'Bookmark'], ['History', 'Bookmark']]


In [21]:
# Define the set of groceries
flattened = [i for t in transactions for i in t]
books = list(set(flattened))

# Generate all possible rules from groceries list
rules = list(permutations(books, 2))

# Print the set of rules
print(rules)

# Print the number of rules
print(len(rules))

[('History', 'Bookmark'), ('History', 'Fiction'), ('History', 'Biography'), ('History', 'Poetry'), ('Bookmark', 'History'), ('Bookmark', 'Fiction'), ('Bookmark', 'Biography'), ('Bookmark', 'Poetry'), ('Fiction', 'History'), ('Fiction', 'Bookmark'), ('Fiction', 'Biography'), ('Fiction', 'Poetry'), ('Biography', 'History'), ('Biography', 'Bookmark'), ('Biography', 'Fiction'), ('Biography', 'Poetry'), ('Poetry', 'History'), ('Poetry', 'Bookmark'), ('Poetry', 'Fiction'), ('Poetry', 'Biography')]
20


## 1.2. The support metric

* One-hot encoding transaction data

In [22]:
# Instantiate transaction encoder and identify unique items in transactions
encoder = TransactionEncoder().fit(transactions)

# One-hot encode transactions
onehot = encoder.transform(transactions)

# Convert one-hot encoded data to DataFrame
onehot = pd.DataFrame(onehot, columns = encoder.columns_)

# Print the one-hot encoded transaction dataset
print(onehot)

    Biography  Bookmark  Fiction  History  Poetry
0       False      True    False     True   False
1       False      True    False     True   False
2       False      True     True    False   False
3        True      True    False    False   False
4       False      True    False     True   False
..        ...       ...      ...      ...     ...
94       True      True    False    False   False
95      False      True    False    False    True
96      False      True    False     True   False
97       True      True    False    False   False
98       True      True    False    False   False

[99 rows x 5 columns]


* Computing the suport metric

In [23]:
# Compute the support
support = onehot.mean()

# Print the support
print(support)

Biography    0.404040
Bookmark     1.000000
Fiction      0.252525
History      0.252525
Poetry       0.090909
dtype: float64


In [24]:
# Add a jam+bread column to the DataFrame onehot
onehot['Biography+Bookmark'] = np.logical_and(onehot['Biography'], onehot['Bookmark'])

# Compute the support
support = onehot.mean()

# Print the support values
print(support)

Biography             0.404040
Bookmark              1.000000
Fiction               0.252525
History               0.252525
Poetry                0.090909
Biography+Bookmark    0.404040
dtype: float64
